# Setup and Imports

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import plotly.express as px
from scipy import stats
import json
import requests
from matplotlib.path import Path as MplPath

# Chatham House Trade data Import

## Importing and merging all trade data files

In [ ]:
# Importing and merging all trade data files

paths = sorted(glob.glob("Input Data/Chatham House Data/Resource Trade Data *.xlsx"))
dfs = [pd.read_excel(p, sheet_name="Trades") for p in paths]
merged = pd.concat(dfs, ignore_index=True).drop_duplicates()
merged = merged.sort_values(["Year"]).reset_index(drop=True)

## Adjust for data quality and presence of sufficient data

In [ ]:
# print(merged.shape)
# print(list(merged.columns))

#Investigate missing values and drop columns where missing values exceed an arbitrary threshold
merged.isna().mean()

MISSING_THRESHOLD = 0.03

missing_frac = merged.isna().mean()
cols_to_keep = missing_frac[missing_frac <= MISSING_THRESHOLD].index

reduced = merged[cols_to_keep].copy()
# print(reduced.shape)

print(f"Dropped columns for not meeting the {100 - MISSING_THRESHOLD*100}% availability threshold:", set(merged.columns) - set(reduced.columns))

In [ ]:
reduced.columns

## Prepare trade data for conversion to network - Aggregate trade values by year and country pairs

In [ ]:
# Aggregate trade values by year and country pairs (indexing columns) - currently the data has two rows per trade (one for each direction)
# Also discard unnecessary columns

INDEXING_COLS=["Year", "Importer", "Importer ISO3", "Exporter", "Exporter ISO3"]
VALUE_COLS=['Value (1000USD)', 'Weight (1000kg)']

pairs_year = (
    reduced
    .groupby(INDEXING_COLS, as_index=False)[VALUE_COLS]
    .sum()
)

# Sort by Year and Total Value descending within each year

pairs_year_sorted = pairs_year.sort_values(
    ["Year", "Value (1000USD)"],
    ascending=[True, False]
)

pairs_year_sorted.shape


In [ ]:
pairs_year_sorted

In [ ]:
pairs_year_sorted.head()

# Trade Concentration Analysis

In [ ]:
def hhi_normalized(series):
    N = len(series)
    if N <= 1:
        return 1.0

    shares = series / series.sum()
    hhi_raw = (shares ** 2).sum()

    # size-adjusted (normalized) HHI
    return (hhi_raw - 1 / N) / (1 - 1 / N)

# Calculate normalized HHI for each year based on trade values and weights

df = pairs_year_sorted.copy()

# ensure numeric
for col in ["Value (1000USD)", "Weight (1000kg)"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.dropna(subset=["Year", "Value (1000USD)", "Weight (1000kg)"])

hhi_by_year = (
    df
    .groupby("Year")
    .agg(
        HHI_value_norm=("Value (1000USD)", hhi_normalized),
        HHI_weight_norm=("Weight (1000kg)", hhi_normalized),
    )
    .reset_index()
)

plt.figure(figsize=(8, 4))

plt.plot(
    hhi_by_year["Year"],
    hhi_by_year["HHI_value_norm"],
    marker="o",
    label="Normalized HHI (Value, 1000 USD)"
)

plt.plot(
    hhi_by_year["Year"],
    hhi_by_year["HHI_weight_norm"],
    marker="s",
    label="Normalized HHI (Weight, 1000 kg)"
)

plt.xlabel("Year")
plt.ylabel("Normalized Herfindahl–Hirschman Index")
plt.title("Global Soy Trade Concentration Over Time")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## Check for correlation between HHI by weight and by dollar value
even though it is visually clear

In [ ]:
#checking if there's a high enough correlation between trade weight and trade value, then we can use one of them for further analysis

rho, p = spearmanr(pairs_year_sorted["Weight (1000kg)"], pairs_year_sorted["Value (1000USD)"])
"{:.2f}".format(float(rho)), "{:.2f}".format(float(p))

# Convert trade data to nodes and edges

In [ ]:
# Prepare Graph from data

df = pairs_year_sorted.copy()
# Rename columns to short, consistent names
df = df.rename(columns={
    "Exporter ISO3": "Exp_ISO3",
    "Importer ISO3": "Imp_ISO3",
    "Value (1000USD)": "Value",
    "Weight (1000kg)": "Weight",
})

# Ensure numeric columns are numeric
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
# df["Weight"] = pd.to_numeric(df["Weight"], errors="coerce")
# using value since it correlates well with weight and is more commonly used in trade network analysis

# Drop rows with missing essentials
df = df.dropna(subset=["Year", "Exporter", "Exp_ISO3", "Imp_ISO3", "Importer", "Value"])

# Remove self-loops (country trading with itself)
self_loops = df[df["Exp_ISO3"] == df["Imp_ISO3"]]
df = df[df["Exp_ISO3"] != df["Imp_ISO3"]]
print(f"Removed {len(self_loops)} self-loops.")

# Build directed graphs for each year
graphs_by_year = {}

for year, dy in df.groupby("Year"):
    G = nx.DiGraph()

    for r in dy.itertuples(index=False):
        if r.Value <= 0:
            continue

        # Add exporter node with attributes
        G.add_node(
            r.Exp_ISO3,
            country=r.Exporter,
            iso3=r.Exp_ISO3,
        )

        # Add importer node with attributes
        G.add_node(
            r.Imp_ISO3,
            country=r.Importer,
            iso3=r.Imp_ISO3,
        )

        # Add edge
        G.add_edge(
            r.Exp_ISO3,
            r.Imp_ISO3,
            weight=r.Value,
            distance=1.0 / r.Value
        )

    graphs_by_year[year] = G

print(f"Constructed graphs for years: {list(graphs_by_year.keys())}")
# Print size of graphs for each year   
for year, G in graphs_by_year.items():
    print(f"Year {year}: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Calculate betweenness centrality for countries in the graph

In [ ]:
betweenness_rows = []

for year, G in graphs_by_year.items():

    bc = nx.betweenness_centrality(
        G,
        weight="distance",
        normalized=True,
    )

    for iso3, score in bc.items():
        betweenness_rows.append({
            "Year": year,
            "ISO3": iso3,
            "Country": G.nodes[iso3].get("country"),  # 👈 pull name from node attrs
            "Betweenness": score,
        })

betweenness_df = pd.DataFrame(betweenness_rows)

print(
    f"Calculated betweenness centrality for {betweenness_df['ISO3'].nunique()} countries for {betweenness_df['Year'].nunique()} years."
)

In [ ]:
betweenness_df.head()

## Calculate slopes for between centrality (for trends)

In [ ]:
def slope_of_series(years, values):
    years = years.astype(float)
    values = values.astype(float)
    if len(values) < 2:
        return np.nan
    return np.polyfit(years, values, 1)[0]

# ensure clean types
bdf = betweenness_df.copy()
bdf["Year"] = pd.to_numeric(bdf["Year"], errors="coerce")
bdf["Betweenness"] = pd.to_numeric(bdf["Betweenness"], errors="coerce")
bdf = bdf.dropna(subset=["Year", "ISO3", "Betweenness"]).copy()
bdf["Year"] = bdf["Year"].astype(int)

betweenness_trend_df = (
    bdf
    .groupby(["ISO3", "Country"], dropna=False)
    .apply(lambda d: pd.Series({
        "Slope": slope_of_series(d["Year"].values, d["Betweenness"].values),
        "MeanBetweenness": d["Betweenness"].mean(),
        "MaxBetweenness": d["Betweenness"].max(),
        "FirstYear": int(d["Year"].min()),
        "LastYear": int(d["Year"].max()),
        "NumYears": int(d["Year"].nunique()),
    }))
    .reset_index()
    .dropna(subset=["Slope"])
)

print(f"Trend rows: {len(betweenness_trend_df):,}")
betweenness_trend_df.sort_values("Slope", ascending=False).head(10)

## Data outputs for graphing and use in report

In [ ]:
# Top 15 by Slope
top15_slope = (
    betweenness_trend_df.sort_values("Slope", ascending=False)
      .loc[:, ["ISO3", "Country", "Slope", "MeanBetweenness", "MaxBetweenness"]]
      .head(15)
)

print("=== Top 15 Countries by Betweenness Slope ===")
display(top15_slope)

# Top 15 by Mean Betweenness
top15_mean_betweenness = (
    betweenness_trend_df.sort_values("MeanBetweenness", ascending=False)
      .loc[:, ["ISO3", "Country", "MeanBetweenness", "Slope", "MaxBetweenness"]]
      .head(15)
)

print("=== Top 15 Countries by Mean Betweenness ===")
display(top15_mean_betweenness)


## Define map styles for chloropleth

In [ ]:
# Define a chloropleth map style

MAP_STYLE = dict(
    projection_type="natural earth",
    showcountries=True,
    showcoastlines=True,
    showland=True,
    showocean=True,
    lataxis_showgrid=False,
    lonaxis_showgrid=False,
)

LAYOUT_STYLE = dict(
    width=1400,
    height=800,
    margin=dict(l=20, r=20, t=60, b=20),
    font=dict(size=14),
)

COLOR_SCALE = "YlOrRd"
COLORBAR_TITLE = "Betweenness slope"


## Use a technical definition for producing country
Otherwise producing countries will also be visible in betweenness data, which is less interesting

In [ ]:
# Identify producing countries:
# - export a lot (absolute scale)
# - import almost nothing
# - strong export >> import asymmetry

MIN_EXPORT_SHARE = 0.03      # total share ≥3% of global exports
MIN_NET_EXPORT_SHARE = 0.02  # net exports ≥2% of global exports
MIN_RATIO = 5               # exports at least 5× imports

df = pairs_year_sorted.copy()

# Keep positive trade only
df = df[
    (df["Weight (1000kg)"] > 0) &
    df["Exporter ISO3"].notna() &
    df["Importer ISO3"].notna()
].copy()

# Total exports and imports per country
exports = df.groupby("Exporter ISO3")["Weight (1000kg)"].sum()
imports = df.groupby("Importer ISO3")["Weight (1000kg)"].sum()

balance = (
    pd.concat([exports, imports], axis=1)
    .fillna(0)
)
balance.columns = ["exports", "imports"]

balance["net_exports"] = balance["exports"] - balance["imports"]
balance["export_import_ratio"] = balance["exports"] / balance["imports"].replace(0, pd.NA)
global_exports = balance["exports"].sum()

# countries that meet all criteria
producing_df = balance[
    (balance["exports"] / global_exports >= MIN_EXPORT_SHARE) &
    (balance["net_exports"] / global_exports >= MIN_NET_EXPORT_SHARE) &
    (balance["export_import_ratio"] >= MIN_RATIO)
].sort_values("exports", ascending=False)

PRODUCING_GEOS = set(producing_df.index)

PRODUCING_GEOS

## Prepare data for plotting

In [ ]:
# Prepare data for plotting: filter to top K rising intermediaries (by slope and mean betweenness), excluding producers
FILTER_TOP_K = True
TOP_K = 15

betweenness_trend_topk = (
    betweenness_trend_df
    .loc[~betweenness_trend_df["ISO3"].isin(PRODUCING_GEOS)] # exclude producers
    .copy()
)

if FILTER_TOP_K:
    top_slope_iso3 = (
        betweenness_trend_topk
        .nlargest(TOP_K, "Slope")["ISO3"]
    )

    top_mean_iso3 = (
        betweenness_trend_topk
        .nlargest(TOP_K, "MeanBetweenness")["ISO3"]
    )

    betweenness_trend_topk["Slope_plot"] = (
        betweenness_trend_topk["Slope"]
        .where(betweenness_trend_topk["ISO3"].isin(top_slope_iso3))
    )

    betweenness_trend_topk["MeanBetweenness_plot"] = (
        betweenness_trend_topk["MeanBetweenness"]
        .where(betweenness_trend_topk["ISO3"].isin(top_mean_iso3))
    )
else:
    betweenness_trend_topk["Slope_plot"] = betweenness_trend_topk["Slope"]
    betweenness_trend_topk["MeanBetweenness_plot"] = betweenness_trend_topk["MeanBetweenness"]

## Generate chloropleth maps for high centrality countries and those with ascending centralities (top K)

In [ ]:
fig1 = px.choropleth(
    betweenness_trend_topk,
    locations="ISO3",
    locationmode="ISO-3",
    color="MeanBetweenness_plot",
    hover_name="Country",
    hover_data={
        "ISO3": True,
        "Slope": ":.3e",
        "MeanBetweenness": ":.3e",
        "MaxBetweenness": ":.3e",
        "NumYears": True,
        "FirstYear": True,
        "LastYear": True,
    },
    color_continuous_scale=COLOR_SCALE,
    title=f"Top {TOP_K} Rising Intermediaries by Mean Betweenness (Producers Omitted)" if FILTER_TOP_K else "Betweenness Trend (Mean Betweenness Over Time, Producers Omitted)"
)

fig1.update_geos(**MAP_STYLE)
fig1.update_layout(**LAYOUT_STYLE)
fig1.update_layout(coloraxis_colorbar_title=COLORBAR_TITLE)

fig1.show()

fig2 = px.choropleth(
    betweenness_trend_topk,
    locations="ISO3",
    locationmode="ISO-3",
    color="Slope_plot",
    hover_name="Country",
    hover_data={
        "ISO3": True,
        "Slope": ":.3e",
        "MeanBetweenness": ":.3e",
        "MaxBetweenness": ":.3e",
        "NumYears": True,
        "FirstYear": True,
        "LastYear": True,
    },
    color_continuous_scale=COLOR_SCALE,
    title=f"Top {TOP_K} Rising Intermediaries by Betweenness Slope (Producers Omitted)" if FILTER_TOP_K else "Betweenness Trend (Slope Over Time, Producers Omitted)"
)

fig2.update_geos(**MAP_STYLE)
fig2.update_layout(**LAYOUT_STYLE)
fig2.update_layout(coloraxis_colorbar_title=COLORBAR_TITLE)

fig2.show()

## Generate a combined version for report

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

df = betweenness_trend_topk.copy()

# Top-15 intermediaries are exactly those with a real Slope_plot value
top15 = df["Slope_plot"].notna()

# Positive slope among top-15
pos_top15 = df.loc[top15 & (df["Slope_plot"] > 0)].copy()

# Base map with choropleth coloring by mean betweenness (only for top-K, if filtering)
fig = px.choropleth(
    df,
    locations="ISO3",
    locationmode="ISO-3",
    color="MeanBetweenness_plot",
    hover_name="Country",
    hover_data={
        "ISO3": True,
        "Slope": ":.3e",
        "MeanBetweenness": ":.3e",
        "MaxBetweenness": ":.3e",
        "NumYears": True,
        "FirstYear": True,
        "LastYear": True,
    },
    color_continuous_scale=COLOR_SCALE,
    title=f"Top {TOP_K}: Mean betweenness (fill) + ▲ if Slope_plot > 0" if FILTER_TOP_K
          else "Mean betweenness (fill) + ▲ if Slope_plot > 0"
)

fig.update_geos(**MAP_STYLE)
fig.update_layout(**LAYOUT_STYLE)
fig.update_layout(coloraxis_colorbar_title="Mean betweenness")

# Overlay positive slope (▲) markers for top-15 intermediaries
fig.add_trace(
    go.Scattergeo(
        locations=pos_top15["ISO3"],
        locationmode="ISO-3",
        mode="markers",
        marker=dict(
            symbol="triangle-up",
            size=10,
            line=dict(width=0.6),
            opacity=0.95
        ),
        name="Positive slope (Top-15 only)",
        text=pos_top15["Country"],
        hovertemplate="<b>%{text}</b><br>ISO3=%{location}<br>Slope_plot=%{customdata:.3e}<extra></extra>",
        customdata=pos_top15["Slope_plot"],
        showlegend=True,
    )
)

fig.show()

fig.write_image("betweenness_combined.png", scale=3)


## Experimental: see if import routes into the Netherlands changed over the last period

In [ ]:
NLD = "NLD"
TOP_K = 10  # top suppliers to track

df = pairs_year_sorted.copy()
value_col = None
for c in ["Value (1000USD)", "Total Value (1000USD)", "value", "Value"]:
    if c in df.columns:
        value_col = c
        break
if value_col is None:
    raise ValueError("Couldn't find a trade value column.")
df[value_col] = pd.to_numeric(df[value_col], errors="coerce")

nld_imports = df[
    (df["Importer ISO3"] == NLD) &
    (df["Exporter ISO3"] != NLD)
].dropna(subset=["Year", "Exporter ISO3", value_col])

top_sets = {}
for year, g in nld_imports.groupby("Year"):
    suppliers = (
        g.groupby("Exporter ISO3")[value_col].sum()
        .sort_values(ascending=False)
        .head(TOP_K)
        .index
        .tolist()
    )
    top_sets[year] = set(suppliers)

years = sorted(top_sets.keys())
rows = []
for i in range(1, len(years)):
    y0, y1 = years[i-1], years[i]
    A, B = top_sets[y0], top_sets[y1]
    jaccard = len(A & B) / len(A | B) if (A | B) else np.nan
    entered = len(B - A)
    exited = len(A - B)
    rows.append({"Year": y1, "JaccardTopK": jaccard, "EnteredTopK": entered, "ExitedTopK": exited})

churn = pd.DataFrame(rows)
print(churn)

plt.figure(figsize=(9, 4))
plt.plot(churn["Year"], churn["JaccardTopK"], marker="o")
plt.xlabel("Year")
plt.ylabel(f"Jaccard similarity of top-{TOP_K} suppliers")
plt.title(f"Netherlands: stability / lock-in of top-{TOP_K} import suppliers")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(9, 4))
plt.plot(churn["Year"], churn["EnteredTopK"], marker="o", label="Entered")
plt.plot(churn["Year"], churn["ExitedTopK"], marker="o", label="Exited")
plt.xlabel("Year")
plt.ylabel(f"Count of suppliers in top-{TOP_K}")
plt.title(f"Netherlands: churn in top-{TOP_K} suppliers")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


## Experimental: Very basic simulation of removal of largest input intermediary, to see which country would substitute.

Not interesting because NL imports directly from Brazil, which is the dominant producer. Mostly trade would be routed through USA, with India and China in two years.

In [ ]:

NLD = "NLD"

# --- import values to identify top supplier per year ---
df = pairs_year_sorted.copy()
value_col = None
for c in ["Value (1000USD)", "Total Value (1000USD)", "value", "Value"]:
    if c in df.columns:
        value_col = c
        break
if value_col is None:
    raise ValueError("Couldn't find a trade value column.")
df[value_col] = pd.to_numeric(df[value_col], errors="coerce")

nld_imports = df[
    (df["Importer ISO3"] == NLD) &
    (df["Exporter ISO3"] != NLD)
].dropna(subset=["Year", "Exporter ISO3", value_col])

imports_by_year = (
    nld_imports
    .groupby(["Year", "Exporter ISO3"])[value_col].sum()
    .reset_index()
)

stress_rows = []

for year, G in graphs_by_year.items():
    if NLD not in G:
        continue

    # NLD betweenness + reachability baseline
    bc = nx.betweenness_centrality(G, weight="distance", normalized=True)
    nld_bc = bc.get(NLD, np.nan)
    base_reach = len(nx.ancestors(G, NLD))  # nodes that can reach NLD

    # top supplier to NLD that year (by direct import value)
    dy = imports_by_year[imports_by_year["Year"] == year]
    if dy.empty:
        continue

    total_import = dy[value_col].sum()
    top_supplier = dy.sort_values(value_col, ascending=False).iloc[0]["Exporter ISO3"]
    top_share = dy.sort_values(value_col, ascending=False).iloc[0][value_col] / total_import

    # top intermediary in the whole graph (highest betweenness excluding NLD itself)
    top_intermediary = max(
        (n for n in bc.keys() if n != NLD),
        key=lambda n: bc[n],
        default=None
    )

    # test removal of top supplier
    GA = G.copy()
    if top_supplier in GA:
        GA.remove_node(top_supplier)
    bcA = nx.betweenness_centrality(GA, weight="distance", normalized=True) if NLD in GA else {}
    reachA = len(nx.ancestors(GA, NLD)) if NLD in GA else 0
    nld_bc_A = bcA.get(NLD, np.nan)

    # test removal of top intermediary
    GB = G.copy()
    if top_intermediary in GB:
        GB.remove_node(top_intermediary)
    bcB = nx.betweenness_centrality(GB, weight="distance", normalized=True) if NLD in GB else {}
    reachB = len(nx.ancestors(GB, NLD)) if NLD in GB else 0
    nld_bc_B = bcB.get(NLD, np.nan)

    stress_rows.append({
        "Year": year,
        "TopSupplier": top_supplier,
        "TopSupplierShareOfNLDImports": top_share,
        "BaseReachToNLD": base_reach,
        "ReachAfterRemoveTopSupplier": reachA,
        "ReachAfterRemoveTopIntermediary": reachB,
        "NLD_Betweenness": nld_bc,
        "NLD_Betweenness_AfterRemoveTopSupplier": nld_bc_A,
        "NLD_Betweenness_AfterRemoveTopIntermediary": nld_bc_B,
        "TopIntermediary": top_intermediary,
        "TopIntermediary_Betweenness": bc.get(top_intermediary, np.nan)
    })

stress_df = pd.DataFrame(stress_rows).sort_values("Year")
display(stress_df)


## Importing Trase Data

In [ ]:
# Importing Trase Data

import pandas as pd
from pathlib import Path

DATA_DIR = Path("Input Data/Trase Import Data")

dfs = []

for file in DATA_DIR.glob("*.xlsx"):
    # read year from the "Filters" sheet, cell C3 on
    filters = pd.read_excel(
        file,
        sheet_name="Filters",
        header=None
    )
    year = int(filters.iloc[2, 2])  # C3 → row 2, col 2 (0-based)

    # read main data from "Data" sheet
    df = pd.read_excel(file, sheet_name="Data")

    df["Year"] = year
    df["SourceFile"] = file.name  # optional but useful

    dfs.append(df)

df_trase = pd.concat(dfs, ignore_index=True)

df_trase.drop(columns=["SourceFile","country_of_first_import"], inplace=True)

df_trase.sample(20)

## Importing Mapbiomas data on deforestation

In [ ]:
# Importing MapBiomas Deforestation Data

import pandas as pd

path = "Input Data/Brazil Deforestation Biome State.xlsx"
deforestation_raw = pd.read_excel(path,sheet_name="DEFORESTATION")

id_cols = [
    "country", "biome", "state", "class", "transition_name",
    "class_level_0", "class_level_1", "class_level_2",
    "class_level_3", "class_level_4"
]

year_cols = [c for c in deforestation_raw.columns if c.isdigit()]

deforestation_long = (
    deforestation_raw
    .melt(
        id_vars=id_cols,
        value_vars=year_cols,
        var_name="year",
        value_name="deforested_area"
    )
)

# optional: make year numeric
deforestation_long["year"] = deforestation_long["year"].astype(int)

deforestation_long[(deforestation_long["year"] == 2020) & (deforestation_long["state"] == "Mato Grosso")]

deforestation_aggregated = (
    deforestation_long
    .groupby(
        ["country", "state", "year"],
        as_index=False
    )["deforested_area"]
    .sum()
)

deforestation_aggregated.drop(columns=["country"], inplace=True)

deforestation_aggregated




In [ ]:
# Merging Deforestation and Trase Data to get a soy panel dataset

defor = deforestation_aggregated.copy()

defor["state"] = defor["state"].str.upper().str.strip()
defor["year"] = defor["year"].astype(int)

# Optional: rename for clarity
defor = defor.rename(columns={"deforested_area": "deforestation_ha"})


soy = df_trase.copy()

soy = soy.query("state != 'UNKNOWN STATE'")
soy["state"] = soy["state"].str.upper().str.strip()
soy["Year"] = soy["Year"].astype(int)

soy = (
    soy
    .groupby(["state", "Year"], as_index=False)
    .agg(soy_volume=("volume", "sum"))
)

state_map = {
    "ACRE": "ACRE",
    "ALAGOAS": "ALAGOAS",
    "AMAPA": "AMAPÁ",
    "AMAZONAS": "AMAZONAS",
    "BAHIA": "BAHIA",
    "CEARA": "CEARÁ",
    "DISTRITO FEDERAL": "DISTRITO FEDERAL",
    "ESPIRITO SANTO": "ESPÍRITO SANTO",
    "GOIAS": "GOIÁS",
    "MARANHAO": "MARANHÃO",
    "MATO GROSSO": "MATO GROSSO",
    "MATO GROSSO DO SUL": "MATO GROSSO DO SUL",
    "MINAS GERAIS": "MINAS GERAIS",
    "PARA": "PARÁ",
    "PARAIBA": "PARAÍBA",
    "PARANA": "PARANÁ",
    "PERNAMBUCO": "PERNAMBUCO",
    "PIAUI": "PIAUÍ",
    "RONDONIA": "RONDÔNIA",
    "RORAIMA": "RORAIMA",
    "SANTA CATARINA": "SANTA CATARINA",
    "SAO PAULO": "SÃO PAULO",
    "SERGIPE": "SERGIPE",
    "TOCANTINS": "TOCANTINS",
}

soy["state_mb"] = soy["state"].map(state_map)
defor["state_mb"] = defor["state"].str.upper()
soy = soy.dropna(subset=["state_mb"])

soy_panel = soy.merge(
    defor,
    left_on=["state_mb", "Year"],
    right_on=["state_mb", "year"],
    how="inner"
).drop(columns=["state_x", "state_y", "year"])

soy_panel.sample(20)


## Define basic State data for Brazil for mergin and further analysis

In [ ]:
# Codes for states for merging
state_to_uf = {
    "ACRE": "AC", "ALAGOAS": "AL", "AMAPÁ": "AP", "AMAZONAS": "AM",
    "BAHIA": "BA", "CEARÁ": "CE", "DISTRITO FEDERAL": "DF",
    "ESPÍRITO SANTO": "ES", "GOIÁS": "GO", "MARANHÃO": "MA",
    "MATO GROSSO": "MT", "MATO GROSSO DO SUL": "MS",
    "MINAS GERAIS": "MG", "PARÁ": "PA", "PARANÁ": "PR",
    "PIAUÍ": "PI", "RONDÔNIA": "RO", "RORAIMA": "RR",
    "SANTA CATARINA": "SC", "SÃO PAULO": "SP", "TOCANTINS": "TO",
    "PARAÍBA": "PB", "PERNAMBUCO": "PE", "RIO DE JANEIRO": "RJ",
    "RIO GRANDE DO NORTE": "RN", "RIO GRANDE DO SUL": "RS", "SERGIPE": "SE",
}

# ensure all states are represented
all_states = pd.DataFrame({
    "uf": [
        "AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS",
        "MG","PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC",
        "SP","SE","TO"
    ]
})

# replace with official file from IBGE?
brazil_states = requests.get(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
).json()

In [ ]:
# Aggregate imports

state_imports = (
    soy_panel
    .groupby("state_mb", as_index=False)
    .agg(mean_soy=("soy_volume", "mean"))   # or "sum"
)

df = state_imports.copy()
df["uf"] = df["state_mb"].map(state_to_uf)

plot_df = all_states.merge(df[["uf", "mean_soy"]], on="uf", how="left")

# Merge areas

areas = pd.read_csv("Input Data/brasil_states_area.csv")
areas["uf"] = areas["uf"].str.upper()

plot_df = plot_df.merge(areas, on="uf", how="left")

# Clculate per area

plot_df["import_density"] = (
    plot_df["mean_soy"] / plot_df["area_km2"]   # imports per km²
)

# log scale for more clarity (otherwise the orders of magnitude difference would throw it off)
plot_df["log_density"] = np.log10(plot_df["import_density"].clip(lower=1e-12))


vmin = plot_df["log_density"].min(skipna=True)
vmax = plot_df["log_density"].max(skipna=True)

sentinel = vmin - 1.0
plot_df["fill"] = plot_df["log_density"].fillna(sentinel) #create a sentinel value in case values are missing. Can't be 0.

eps = 1e-6
grey = "#e6e6e6"
reds = px.colors.sequential.Reds

colorscale = [[0.0, grey], [eps, grey]]
for i, c in enumerate(reds):
    t = eps + (1 - eps) * (i / (len(reds) - 1))
    colorscale.append([t, c])

# Chloropleth
fig = px.choropleth(
    plot_df,
    geojson=brazil_states,
    locations="uf",
    featureidkey="properties.sigla",
    color="fill",
    color_continuous_scale=colorscale,
    range_color=(sentinel, vmax),
    hover_data={
        "uf": True,
        "mean_soy": ":,.0f",
        "area_km2": ":,.0f",
        "import_density": ":.3f",
        "log_density": False,
        "fill": False,
    },
    title="NL Soy Import Intensity by Brazilian State (imports per km²)",
)

fig.update_geos(
    visible=False,
    scope="south america",
    projection_type="mercator",
    lonaxis_range=[-75, -30],
    lataxis_range=[-35, 6],
)

fig.update_traces(marker_line_color="grey", marker_line_width=0.6)

fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="log₁₀ imports / km²"),
)

fig.show()

fig.write_image("brazil_soy_import_density.png", scale=3)


## Soy imports to NL vs deforestation

In [ ]:

# Change lag to test other hypothesis - no clear theory
LAG_YRS = 0 

df = soy_panel.sort_values(["state_mb", "Year"]).copy()

# lag within state
df["soy_lag"] = df.groupby("state_mb")["soy_volume"].shift(LAG_YRS)

# drop missing
df = df.dropna(subset=["soy_lag", "deforestation_ha"])

# optional but recommended: log-log
x = np.log1p(df["soy_lag"].to_numpy())
y = np.log1p(df["deforestation_ha"].to_numpy())

# Regression
X = np.column_stack([np.ones_like(x), x])         # [1, x]
beta, *_ = np.linalg.lstsq(X, y, rcond=None)      # [intercept, slope]
intercept, slope = beta

y_hat = intercept + slope * x
ss_res = np.sum((y - y_hat) ** 2)
ss_tot = np.sum((y - y.mean()) ** 2)
r2 = 1 - ss_res / ss_tot

p_value = None

n = len(x)
se2 = ss_res / (n - 2)
sxx = np.sum((x - x.mean()) ** 2)
se_slope = np.sqrt(se2 / sxx)
t = slope / se_slope
p_value = 2 * (1 - stats.t.cdf(abs(t), df=n - 2))


plt.figure(figsize=(6, 4))
plt.scatter(x, y, alpha=0.6)

x_line = np.linspace(x.min(), x.max(), 100)
plt.plot(x_line, intercept + slope * x_line)

plt.xlabel(f"log(soy imports + 1), lag = {LAG_YRS}")
plt.ylabel("log(deforestation + 1)")
plt.title(f"Soy imports vs deforestation (lag {LAG_YRS} years)")
plt.tight_layout()
plt.show()


print(f"n = {len(x)}")
print(f"log(deforestation+1) = {intercept:.4f} + {slope:.4f} * log(soy+1)")
print(f"R² = {r2:.4f}")
if p_value is not None:
    print(f"p-value (slope) = {p_value:.3g}")
else:
    print("p-value (slope) = (scipy not available / not compatible)")



## Experimental: Calculate soy import (to NL) exposure by state of production
Not used for report because of incomplete import data, and dominance of import source over deforestation metrics.

In [ ]:
state_exposure = (
    soy_panel
    .groupby("state_mb", as_index=False)
    .agg(
        mean_soy=("soy_volume", "mean"),
        mean_defor=("deforestation_ha", "mean"),
    )
)

state_exposure["exposure_score"] = state_exposure["mean_soy"] * state_exposure["mean_defor"]

df = state_exposure.copy()
df["uf"] = df["state_mb"].map(state_to_uf)

plot_df = all_states.merge(df, on="uf", how="left")

areas["uf"] = areas["uf"].str.upper()

plot_df = plot_df.merge(areas, on="uf", how="left").drop(columns=["state_mb"])

# Expecting areas has area_km2; convert to hectares
plot_df["area_ha"] = plot_df["area_km2"] * 100.0

# Intensity: deforestation per unit state area (ha / ha)
plot_df["defor_intensity"] = plot_df["mean_defor"] / plot_df["area_ha"]

# Exposure adjusted by intensity (soy × deforestation intensity)
plot_df["exposure_intensity"] = plot_df["mean_soy"] * plot_df["defor_intensity"]

# Now log-scale the adjusted exposure
plot_df["log_exposure"] = np.log10(plot_df["exposure_intensity"].clip(lower=1e-12))

# Decompose exposure into additive log components
plot_df["log_imports"] = np.log10(plot_df["mean_soy"].clip(lower=1e-12))
plot_df["log_defor_intensity"] = np.log10(plot_df["defor_intensity"].clip(lower=1e-12))

# check for math errors
plot_df["log_exposure_check"] = (
    plot_df["log_imports"] + plot_df["log_defor_intensity"]
)

vmin = plot_df["log_exposure"].min(skipna=True)
vmax = plot_df["log_exposure"].max(skipna=True)

if pd.isna(vmin) or pd.isna(vmax):
    vmin, vmax = 0.0, 1.0

sentinel = vmin - 1.0
plot_df["fill"] = plot_df["log_exposure"].fillna(sentinel)

eps = 1e-6
grey = "#e6e6e6"
reds = px.colors.sequential.Reds

colorscale = [
    [0.0, grey],
    [eps, grey],
]
for i, c in enumerate(reds):
    t = eps + (1 - eps) * (i / (len(reds) - 1))
    colorscale.append([t, c])

fig = px.choropleth(
    plot_df,
    geojson=brazil_states,
    locations="uf",
    featureidkey="properties.sigla",
    color="fill",
    color_continuous_scale=colorscale,
    range_color=(sentinel, vmax),
    hover_name="state_name",
    hover_data={
        "mean_soy": ":,.0f",
        "mean_defor": ":,.0f",
        "defor_intensity": ":.3e",
        "exposure_intensity": ":.3e",
        "uf": True,
        "log_exposure": False,
        "fill": False,
    },
    title="NL Soy Import Exposure to Deforestation by Brazilian State (Area-normalised)",
)

fig.update_geos(
    visible=False,
    scope="south america",
    projection_type="mercator",
    lonaxis_range=[-75, -30],
    lataxis_range=[-35, 6],
)

fig.update_traces(marker_line_color="black", marker_line_width=0.6)

fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="log₁₀ exposure"),
)

fig.show()

fig.write_image("brazil_soy_exposure.png", scale=3)

# Climate Soy Yields changes

In [ ]:
CLIMATE_PATH = "Input Data/Soy Yield Brazil 3.5 vs 1.5 Annual.csv"

df = pd.read_csv(CLIMATE_PATH, skiprows=8)

print("Raw preview:")
display(df.head())

# first column = latitude
df = df.rename(columns={df.columns[0]: "lat"})

# longitude values come from column names
lon_vals = np.array([float(c) for c in df.columns[1:]])
lat_vals = pd.to_numeric(df["lat"]).to_numpy()

# grid values
Z = df.iloc[:, 1:].astype(float).to_numpy()

print("Lat long grid shape", Z.shape)
print ("Lat long range:")
print("Lon range:", lon_vals.min(), "→", lon_vals.max())
print("Lat range:", lat_vals.min(), "→", lat_vals.max())
print("Range of pp changes (for validation):", np.nanmin(Z), "→", np.nanmax(Z))


## Plotting of states by climate change impact

In [ ]:
# reusing plot_df from previous exposure calculation
exposure_col = "log_exposure" if "log_exposure" in plot_df.columns else "fill"

# Map bounds wirh margin
LON_RANGE = (-75, -30)
LAT_RANGE = (-35, 6)


clim = pd.read_csv(CLIMATE_PATH, skiprows=8)
clim = clim.rename(columns={clim.columns[0]: "lat"})

lon = np.array([float(c) for c in clim.columns[1:]], dtype=float)
lat = pd.to_numeric(clim["lat"], errors="coerce").to_numpy(dtype=float)
Z = clim.iloc[:, 1:].apply(pd.to_numeric, errors="coerce").to_numpy(dtype=float)

# sort by latitude for nicer display
order = np.argsort(lat)
lat = lat[order]
Z = Z[order, :]

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x=lon,
        y=lat,
        z=Z,
        zsmooth="best",

        colorscale=[
            [0.0,  "#d73027"],   
            [0.5,  "#ffffff"],
            [1.0,  "#1a9850"], 
        ],

        zmid=0,  # ← makes white exactly zero-centered

        hovertemplate="lon=%{x}<br>lat=%{y}<br>climate=%{z:.3f}<extra></extra>",
        colorbar=dict(title="Climate impact"),
        name="Climate grid"
    )
)


brazil_states = requests.get(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
).json()

def add_polygon_lines(coords, name=None):
    """
    coords: either Polygon ring list or MultiPolygon list of polygons
    Adds outlines as go.Scatter line traces (cartesian).
    """
    if not coords:
        return

    is_multipolygon = isinstance(coords[0][0][0], (list, tuple))

    polygons = coords if is_multipolygon else [coords]

    for poly in polygons:
        # outer ring is poly[0]
        ring = poly[0]
        xs = [p[0] for p in ring]
        ys = [p[1] for p in ring]
        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                mode="lines",
                line=dict(width=1, color="rgba(120,120,120,0.7)"),
                name=name if name else "State border",
                showlegend=False,
                hoverinfo="skip"
            )
        )

# draw all states
for feat in brazil_states["features"]:
    geom = feat["geometry"]
    name = feat["properties"].get("sigla", None)
    add_polygon_lines(geom["coordinates"], name=name)


# Plot like a map
fig.update_xaxes(
    title="Longitude",
    range=list(LON_RANGE),
    showgrid=False, # remove gridlines
    zeroline=False
)

fig.update_yaxes(
    title="Latitude",
    range=list(LAT_RANGE),
    scaleanchor="x",
    scaleratio=1,
    showgrid=False,
    zeroline=False
)

fig.update_layout(
    title="Percentage point impact of climate change on soy yield in Brazil with 3.5°C vs 1.5°C warming",
    margin=dict(l=10, r=10, t=50, b=10),
    height=800,
    plot_bgcolor="white",
    paper_bgcolor="white"
)

fig.show()

fig.write_image("brazil_soy_yields_climate_impact.png", scale=3)

In [ ]:
# build mesh of cell centers
LON, LAT = np.meshgrid(lon, lat)
points = np.column_stack([LON.ravel(), LAT.ravel()])

rows = []

for feat in brazil_states["features"]:
    uf = feat["properties"]["sigla"]
    geom = feat["geometry"]["coordinates"]

    if feat["geometry"]["type"] == "Polygon":
        rings = [geom[0]]
    else:
        rings = [poly[0] for poly in geom]

    mask = np.zeros(len(points), dtype=bool)

    for ring in rings:
        path = MplPath(ring)
        mask |= path.contains_points(points)

    state_vals = Z.ravel()[mask]

    mean_change_pp = np.nan if len(state_vals) == 0 else np.nanmean(state_vals)

    rows.append({
        "uf": uf,
        "mean_climate_change_pp": mean_change_pp
    })

climate_state = pd.DataFrame(rows)

result = plot_df.merge(climate_state, on="uf", how="left")

# drop states with missing data
result = result.dropna(subset=["mean_soy", "mean_climate_change_pp"]).copy()

result["mean_climate_change"] = result["mean_climate_change_pp"] / 100.0

result["adjusted_imports"] = (
    result["mean_soy"] * (1 + result["mean_climate_change"])
)

result["import_change"] = (
    result["adjusted_imports"] - result["mean_soy"]
)

display(
    result[[
        "uf",
        "mean_soy",
        "mean_climate_change_pp",
        "mean_climate_change",
        "adjusted_imports",
        "import_change"
    ]].sort_values("mean_soy", ascending=False)
)

In [ ]:
baseline_total = result["mean_soy"].sum()
adjusted_total = result["adjusted_imports"].sum()

shock_abs = adjusted_total - baseline_total
shock_rel = shock_abs / baseline_total

summary = pd.DataFrame({
    "baseline_imports": [baseline_total],
    "adjusted_imports": [adjusted_total],
    "absolute_shock": [shock_abs],
    "relative_shock_frac": [shock_rel],
    "relative_shock_percent": [shock_rel * 100],
})

display(summary)


## Experimental: Simulate change in import because of production/yield adjustments under the climate change scenario

In [ ]:


# input DF should have: uf, mean_soy, mean_climate_change_pp, adjusted_imports, import_change
df = result.copy()

plot_df = all_states.merge(df, on="uf", how="left")

# plotting projected import change
value_col = "import_change" 

vmin = plot_df[value_col].min(skipna=True)
vmax = plot_df[value_col].max(skipna=True)

if pd.isna(vmin) or pd.isna(vmax):
    vmin, vmax = -1.0, 1.0

absmax = max(abs(vmin), abs(vmax))
vmin, vmax = -absmax, absmax

# sentinel for missing data -> grey
sentinel = vmin - absmax * 0.05 - 1.0
plot_df["fill"] = plot_df[value_col].fillna(sentinel)

# colors definition: grey for missing, then red-white-green diverging
eps = 1e-6
grey = "#e6e6e6"
rdbu = px.colors.diverging.RdBu  # red-blue diverging

colorscale = [
    [0.0, grey],
    [eps, grey],
]
for i, c in enumerate(rdbu):
    t = eps + (1 - eps) * (i / (len(rdbu) - 1))
    colorscale.append([t, c])

# ---- geojson ----
brazil_states = requests.get(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
).json()

# ---- map ----
fig = px.choropleth(
    plot_df,
    geojson=brazil_states,
    locations="uf",
    featureidkey="properties.sigla",
    color="fill",
    color_continuous_scale=colorscale,
    range_color=(sentinel, vmax),
    hover_name="uf",
    hover_data={
        "mean_soy": ":,.0f",
        "mean_climate_change_pp": ":.2f",
        "adjusted_imports": ":,.0f",
        "import_change": ":,.0f",
        "uf": True,
        "fill": False,
    },
    title="NL Soy Imports from Brazil — Climate-adjusted Import Change by State",
)

fig.update_geos(
    visible=False,
    scope="south america",
    projection_type="mercator",
    lonaxis_range=[-75, -30],
    lataxis_range=[-35, 6],
)

fig.update_traces(marker_line_color="black", marker_line_width=0.6)

fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Import change (tonnes)"),
)

fig.show()